In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [3]:
with open("/data2/elilouis/sublimationofsnow/thermistor_harp/202303061845.log") as f:
    lines = f.readlines()

In [4]:
cleaned_lines = [f for f in lines if f.startswith('TS302') or f.startswith('TS301')]
delimited_lines = [line.split() for line in cleaned_lines]
temp_only_lines = [[token for token in line if token.startswith('TS') or '.' in token] for line in delimited_lines]
df = pd.DataFrame(temp_only_lines).drop(columns=5)

In [5]:
low_df = df[df[0]=='TS302']
low_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(low_df))
low_df = low_df.rename(columns={1:5, 2:15, 3:25, 4:35})
low_df = low_df.drop(columns=0)
low_df['side'] = 'low'
low_df

/tmp/ipykernel_152849/2556209242.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(low_df))


,5,15,25,35,time_seconds,side
1,16.944915,17.153961,17.432312,17.592132,0,low
3,16.980773,17.245513,17.546936,17.663543,5,low
5,16.976379,17.275604,17.595367,17.692565,10,low
7,17.116088,17.380279,17.640228,17.679077,15,low
9,17.200195,17.444854,17.627502,17.651275,20,low
...,...,...,...,...,...,...
2775,-3.982178,-1.814819,-1.664642,-1.240845,6935,low
2777,-4.065369,-1.995666,-1.873657,-1.392700,6940,low
2779,-4.324921,-2.419067,-2.256592,-1.676178,6945,low
2781,-4.351562,-2.666138,-2.443817,-1.991730,6950,low


In [6]:
high_df = df[df[0]=='TS301']
high_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(high_df))
high_df = high_df.rename(columns={1:10, 2:20, 3:30, 4:40})
high_df = high_df.drop(columns=0)
high_df['side'] = 'high'
high_df

/tmp/ipykernel_152849/40103918.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_df.loc[:, 'time_seconds'] = 5*np.arange(0, len(high_df))


,10,20,30,40,time_seconds,side
0,16.279998,16.555023,16.766693,17.102600,0,high
2,16.414337,16.697448,16.885498,17.156585,5,high
4,16.496368,16.755554,16.935485,17.203155,10,high
6,16.397491,16.635009,16.887725,17.157684,15,high
8,16.337341,16.571472,16.852355,17.135284,20,high
...,...,...,...,...,...,...
2774,-2.233948,-1.500854,-1.392487,-1.399078,6935,high
2776,-2.282654,-1.586090,-1.494232,-1.527069,6940,high
2778,-2.445190,-1.816223,-1.777649,-1.843842,6945,high
2780,-2.548065,-2.148468,-2.061127,-2.156677,6950,high


In [7]:
df = pd.concat([
    low_df.melt(id_vars=['time_seconds', 'side']),
    high_df.melt(id_vars=['time_seconds', 'side'])
]).sort_values(['time_seconds', 'variable'])

In [8]:
df['variable'] = df['variable'] - 5

In [10]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [22]:
alt.Chart(df.tail(8000)).transform_window(
    rolling_mean='mean(value)',
    frame=[60,60],
    groupby=['variable']
).mark_line().encode(
    alt.X('time_seconds:Q'),
    alt.Y('rolling_mean:Q', scale=alt.Scale(zero=False)),
    alt.Color('variable:O', scale = alt.Scale(scheme='turbo'))
).properties(width=1000)

alt.Chart(...)

In [14]:
df['value'] = df['value'].astype(float)

In [15]:
alt.Chart(df.query("time_seconds > 185")).mark_line(point=True).encode(
    alt.X('value:Q', sort='-y'),
    alt.Y('variable:Q'),
    alt.Facet('time_seconds:Q')
).properties(8width=100)

alt.Chart(...)

In [12]:


alt.Chart(
    df.query("time_seconds > 185").groupby("variable").mean().reset_index()
).mark_line(point=True).encode(
    alt.X('value:Q', sort='-y', title='Temperature (˚C)'),
    alt.Y('variable:Q', title='Height above snow (mm)')
).properties(width=100)

/tmp/ipykernel_120094/3984453817.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.query("time_seconds > 185").groupby("variable").mean().reset_index()
/home/elilouis/mambaforge/envs/arm/lib/python3.11/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)